In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.8 MB/s eta 0:00:00


In [3]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296186 sha256=4be46e689f5f5bd87b2821ad437f5632746c3d5abc75a8f747a044f38ff40e66
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [6]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from gensim.models import FastText

# Mount Google Drive
drive.mount('/content/drive')

# Function to load and preprocess data, including FastText embeddings
def process_features_fasttext(input_file, text_column, numeric_features):
    df = pd.read_excel(input_file)
    tokenized_texts = df[text_column].astype(str).apply(lambda x: x.split())
    fasttext_model = FastText(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4, sg=1)
    text_embeddings = tokenized_texts.apply(
        lambda tokens: np.mean([fasttext_model.wv[token] for token in tokens if token in fasttext_model.wv]
                               or [np.zeros(fasttext_model.vector_size)], axis=0)
    )
    text_embeddings = np.stack(text_embeddings.values)
    numeric_df = df[numeric_features].reset_index(drop=True)
    combined_features = np.hstack([text_embeddings, numeric_df.values])
    return combined_features, df

# Apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    from sklearn.decomposition import PCA
    pca = PCA()
    X_reduced = pca.fit_transform(X)
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    return random_search.best_estimator_

# Model functions
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'activation': ['relu', 'tanh'], 'solver': ['adam']}
    model = tune_model(MLPClassifier(max_iter=500), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]}
    model = tune_model(XGBClassifier(use_label_encoder=False, eval_metric='logloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}
    model = tune_model(AdaBoostClassifier(), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
    model = tune_model(LogisticRegression(), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and features
input_file = '/content/drive/MyDrive/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']
X, original_data = process_features_fasttext(input_file, text_column, numeric_features)
X_pca, pca_model = apply_pca(X)
y = original_data['confidence'].apply(lambda x: x + 1)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


KNN Model Metrics:
Train Accuracy: 0.6975
Train Precision: 0.6351
Train Recall: 0.6975
Train F1 Score: 0.6409
Train Confusion Matrix:
[[  0   6  26]
 [  0  39  93]
 [  0  19 293]]

Test Accuracy: 0.6341
Test Precision: 0.5771
Test Recall: 0.6341
Test F1 Score: 0.6039
Test Confusion Matrix:
[[  0   2  11]
 [  0   7  34]
 [  0  28 123]]

Running Naive Bayes model...
Naive Bayes Model Metrics:
Train Accuracy: 0.6471
Train Precision: 0.5056
Train Recall: 0.6471
Train F1 Score: 0.5262
Train Confusion Matrix:
[[  0   1  31]
 [  0   3 129]
 [  0   7 305]]

Test Accuracy: 0.7171
Test Precision: 0.5737
Test Recall: 0.7171
Test F1 Score: 0.6230
Test Confusion Matrix:
[[  0   0  13]
 [  0   1  40]
 [  0   5 146]]

Running MLP model...
MLP Model Metrics:
Train Accuracy: 0.6597
Train Precision: 0.5860
Train Recall: 0.6597
Train F1 Score: 0.5677
Train Confusion Matrix:
[[  0   0  32]
 [  0  15 117]
 [  0  13 299]]

Test Accuracy: 0.6927
Test Precision: 0.5535
Test Recall: 0.6927
Test F1 Score: 0.609

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Random Forest Model Metrics:
Train Accuracy: 0.8992
Train Precision: 0.9084
Train Recall: 0.8992
Train F1 Score: 0.8916
Train Confusion Matrix:
[[ 15   4  13]
 [  0 101  31]
 [  0   0 312]]

Test Accuracy: 0.6293
Test Precision: 0.6339
Test Recall: 0.6293
Test F1 Score: 0.6289
Test Confusion Matrix:
[[  1   5   7]
 [  0  14  27]
 [  4  33 114]]

Running Decision Tree model...


...

XGBoost Model Metrics:
Train Accuracy: 0.8004
Train Precision: 0.7944
Train Recall: 0.8004
Train F1 Score: 0.7812
Train Confusion Matrix:
[[  6   8  18]
 [  1  82  49]
 [  1  18 293]]

Test Accuracy: 0.6537
Test Precision: 0.6512
Test Recall: 0.6537
Test F1 Score: 0.6422
Test Confusion Matrix:
[[  1   3   9]
 [  0  13  28]
 [  1  30 120]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Model Metrics:
Train Accuracy: 0.7017
Train Precision: 0.7004
Train Recall: 0.7017
Train F1 Score: 0.6476
Train Confusion Matrix:
[[  3   8  21]
 [  0  35  97]
 [  0  16 296]]

Test Accuracy: 0.6683
Test Precision: 0.5650
Test Recall: 0.6683
Test F1 Score: 0.6086
Test Confusion Matrix:
[[  0   1  12]
 [  0   3  38]
 [  0  17 134]]

Running CatBoost model...
CatBoost Model Metrics:
Train Accuracy: 0.7962
Train Precision: 0.8052
Train Recall: 0.7962
Train F1 Score: 0.7679
Train Confusion Matrix:
[[  2  11  19]
 [  0  82  50]
 [  0  17 295]]

Test Accuracy: 0.6049
Test Precision: 0.5908
Test Recall: 0.6049
Test F1 Score: 0.5969
Test Confusion Matrix:
[[  0   3  10]
 [  0  12  29]
 [  0  39 112]]

Running Logistic Regression model...
Logistic Regression Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Reca

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]



In [8]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from gensim.models import FastText

# Mount Google Drive
drive.mount('/content/drive')

# Function to process features including FastText embeddings and numeric features
def process_features_fasttext(input_file, text_column, numeric_features):
    # Load the dataset
    df = pd.read_excel(input_file)

    # Train a FastText model on the text column
    sentences = df[text_column].astype(str).apply(lambda x: x.split())
    fasttext_model = FastText(sentences=sentences, vector_size=100, window=5, min_count=1, sg=1, workers=4)
    print("FastText model training completed.")

    # Compute FastText embeddings for each text entry
    text_embeddings = np.array([
        np.mean([fasttext_model.wv[word] for word in sentence if word in fasttext_model.wv] or [np.zeros(100)], axis=0)
        for sentence in sentences
    ])
    print(f"FastText embedding matrix dimensions: {text_embeddings.shape}")

    # Extract numeric features and combine with text embeddings
    numeric_df = df[numeric_features]
    embedding_df = pd.DataFrame(text_embeddings, columns=[f"embedding_{i}" for i in range(text_embeddings.shape[1])])

    combined_df = pd.concat([embedding_df, numeric_df.reset_index(drop=True)], axis=1)

    return combined_df, df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")


def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40],
        'min_samples_split': [2, 5, 10, 15],
        'min_samples_leaf': [1, 2, 4]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and feature details
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'
numeric_features = ['filler_ratio', 'avg_word_length', 'sentence_length_variation']

# Process features
X, original_data = process_features_fasttext(input_file, text_column, numeric_features)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
FastText model training completed.
FastText embedding matrix dimensions: (681, 100)
Number of components selected: 1
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 4}
KNN Model Metrics:
Train Accuracy: 0.7458
Train Precision: 0.7678
Train Recall: 0.7458
Train F1 Score: 0.7278
Train Confusion Matrix:
[[  2  10  20]
 [  0  93  39]
 [  0  52 260]]

Test Accuracy: 0.6195
Test Precision: 0.6621
Test Recall: 0.6195
Test F1 Score: 0.6279
Test Confusion Matrix:
[[  0   4   9]
 [  1  25  15]
 [  2  47 102]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.8887
Train Precision: 0.8959
Train Recall: 0.8887
Train F1 Score: 0.8812
Train Confusion Matrix:
[[ 15   4  13]
 [  0 100  32]
 [  0   4 308]]

Test Accuracy: 0.6098
Test Precision: 0.6042

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (50,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]

Running XGBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.7731
Train Precision: 0.7530
Train Recall: 0.7731
Train F1 Score: 0.7476
Train Confusion Matrix:
[[  3   9  20]
 [  1  75  56]
 [  2  20 290]]

Test Accuracy: 0.6488
Test Precision: 0.6150
Test Recall: 0.6488
Test F1 Score: 0.6314
Test Confusion Matrix:
[[  0   2  11]
 [  0  13  28]
 [  2  29 120]]

Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 100, 'depth': 6}
CatBoost Model Metrics:
Train Accuracy: 0.7458
Train Precision: 0.7601
Train Recall: 0.7458
Train F1 Score: 0.7047
Train Confusion Matrix:
[[  2   6  24]
 [  0  55  77]
 [  0  14 298]]

Test Accuracy: 0.7073
Test Precision: 0.6490
Test Recall: 0.7073
Test F1 Score: 0.6733
Test Confusion Matrix:
[[  0   1  12]
 [  1  12  28]
 [  2  16 133]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
Logistic Regression Model Metrics:
Train Accuracy: 0.6597
Train Precision: 0.7088
Train Recall: 0.6597
Train F1 Score: 0.5286
Train Confusion Matrix:
[[  0   0  32]
 [  0   2 130]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'linear', 'C': 0.1}
SVM Model Metrics:
Train Accuracy: 0.6555
Train Precision: 0.4296
Train Recall: 0.6555
Train F1 Score: 0.5190
Train Confusion Matrix:
[[  0   0  32]
 [  0   0 132]
 [  0   0 312]]

Test Accuracy: 0.7366
Test Precision: 0.5426
Test Recall: 0.7366
Test F1 Score: 0.6249
Test Confusion Matrix:
[[  0   0  13]
 [  0   0  41]
 [  0   0 151]]



In [10]:
!pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2024-11-17 14:03:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.238.176.126, 18.238.176.44, 18.238.176.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.238.176.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   118MB/s    in 44s     

2024-11-17 14:04:06 (97.5 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [17]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import fasttext  # fastText import
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert text column to fastText embeddings
def convert_to_fasttext(input_file, text_column, model_path=None):
    df = pd.read_excel(input_file)

    # Load fastText model if provided (otherwise you can use pretrained)
    if model_path:
        model = fasttext.load_model(model_path)
    else:
        model = fasttext.load_model('cc.en.300.bin')  # Pre-trained model (English)

    # Get fastText embeddings for each text in the column
    def get_fasttext_embedding(text):
        words = text.split()
        word_vectors = np.array([model.get_word_vector(word) for word in words if word in model])
        return np.mean(word_vectors, axis=0) if len(word_vectors) > 0 else np.zeros(300)  # Size of embeddings

    # Apply fastText embeddings to the text column
    df['fasttext_embeddings'] = df[text_column].apply(lambda x: get_fasttext_embedding(str(x)))

    # Convert the embeddings into a DataFrame
    embeddings = np.array(df['fasttext_embeddings'].to_list())
    return pd.DataFrame(embeddings), df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'cleaned_answer'

# Convert text column to fastText embeddings and get the original DataFrame
fasttext_data, original_data = convert_to_fasttext(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(fasttext_data, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 53
Running KNN model...
Best Params: {'weights': 'distance', 'n_neighbors': 7}
KNN Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.7073
Test Precision: 0.6482
Test Recall: 0.7073
Test F1 Score: 0.6654
Test Confusion Matrix:
[[  1   0  12]
 [  2   8  31]
 [  1  14 136]]

Running Random Forest model...
Best Params: {'n_estimators': 150, 'min_samples_split': 10, 'max_depth': 10}
Random Forest Model Metrics:
Train Accuracy: 0.9496
Train Precision: 0.9515
Train Recall: 0.9496
Train F1 Score: 0.9469
Train Confusion Matrix:
[[ 20   3   9]
 [  0 121  11]
 [  0   1 311]]

Test Accuracy: 0.7171
Test Precision: 0.5737
Test Recall: 0.7171
Test F1 Score: 0.6230
Test Confusion Matrix:
[[  0   0  13]

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8466
Train Precision: 0.8501
Train Recall: 0.8466
Train F1 Score: 0.8362
Train Confusion Matrix:
[[ 16   8   8]
 [  0  83  49]
 [  1   7 304]]

Test Accuracy: 0.7073
Test Precision: 0.6743
Test Recall: 0.7073
Test F1 Score: 0.6874
Test Confusion Matrix:
[[  1   0  12]
 [  2  15  24]
 [  2  20 129]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.1}
XGBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6439
Test Precision: 0.6026
Test Recall: 0.6439
Test F1 Score: 0.6224
Test Confusion Matrix:
[[  0   2  11]
 [  1  10  30]
 [  2  27 122]]

Running AdaBoost model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 150, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7437
Train Precision: 0.7374
Train Recall: 0.7437
Train F1 Score: 0.7360
Train Confusion Matrix:
[[ 21   5   6]
 [  0  63  69]
 [  1  41 270]]

Test Accuracy: 0.6488
Test Precision: 0.6438
Test Recall: 0.6488
Test F1 Score: 0.6453
Test Confusion Matrix:
[[  0   6   7]
 [  1  16  24]
 [  4  30 117]]

Running CatBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.9769
Train Precision: 0.9773
Train Recall: 0.9769
Train F1 Score: 0.9767
Train Confusion Matrix:
[[ 31   0   1]
 [  0 123   9]
 [  0   1 311]]

Test Accuracy: 0.6390
Test Precision: 0.6236
Test Recall: 0.6390
Test F1 Score: 0.6258
Test Confusion Matrix:
[[  1   4   8]
 [  0  10  31]
 [  2  29 120]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.6933
Train Precision: 0.6840
Train Recall: 0.6933
Train F1 Score: 0.6385
Train Confusion Matrix:
[[  7   5  20]
 [  0  28 104]
 [  0  17 295]]

Test Accuracy: 0.6878
Test Precision: 0.6009
Test Recall: 0.6878
Test F1 Score: 0.6353
Test Confusion Matrix:
[[  0   1  12]
 [  0   6  35]
 [  1  15 135]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8130
Train Precision: 0.8259
Train Recall: 0.8130
Train F1 Score: 0.7930
Train Confusion Matrix:
[[ 19   5   8]
 [  1  61  70]
 [  2   3 307]]

Test Accuracy: 0.6878
Test Precision: 0.6409
Test Recall: 0.6878
Test F1 Score: 0.6581
Test Confusion Matrix:
[[  1   1  11]
 [  2   9  30]
 [  5  15 131]]



In [11]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import fasttext  # fastText import
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Function to convert text column to fastText embeddings
def convert_to_fasttext(input_file, text_column, model_path=None):
    df = pd.read_excel(input_file)

    # Load fastText model if provided (otherwise you can use pretrained)
    if model_path:
        model = fasttext.load_model(model_path)
    else:
        model = fasttext.load_model('cc.en.300.bin')  # Pre-trained model (English)

    # Get fastText embeddings for each text in the column
    def get_fasttext_embedding(text):
        words = text.split()
        word_vectors = np.array([model.get_word_vector(word) for word in words if word in model])
        return np.mean(word_vectors, axis=0) if len(word_vectors) > 0 else np.zeros(300)  # Size of embeddings

    # Apply fastText embeddings to the text column
    df['fasttext_embeddings'] = df[text_column].apply(lambda x: get_fasttext_embedding(str(x)))

    # Convert the embeddings into a DataFrame
    embeddings = np.array(df['fasttext_embeddings'].to_list())
    return pd.DataFrame(embeddings), df

# Function to apply PCA with automatic component selection based on explained variance
def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def catboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'iterations': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.3], 'depth': [4, 6, 10]}
    model = tune_model(CatBoostClassifier(verbose=0), param_grid, X_train, y_train)
    print("CatBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'CatBoost': catboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file and text column
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_PREPROCESSED.xlsx'
text_column = 'answer'

# Convert text column to fastText embeddings and get the original DataFrame
fasttext_data, original_data = convert_to_fasttext(input_file, text_column)

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(fasttext_data, variance_threshold=0.95)  # Adjust variance threshold as needed

# Prepare target variable
y = original_data['confidence'].apply(lambda x: x + 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of components selected: 57
Running KNN model...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Params: {'weights': 'distance', 'n_neighbors': 14}
KNN Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9834
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 127   5]
 [  0   1 311]]

Test Accuracy: 0.7220
Test Precision: 0.6824
Test Recall: 0.7220
Test F1 Score: 0.6859
Test Confusion Matrix:
[[  1   2  10]
 [  0  11  30]
 [  1  14 136]]

Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': None}
Random Forest Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9834
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 127   5]
 [  0   1 311]]

Test Accuracy: 0.7122
Test Precision: 0.6120
Test Recall: 0.7122
Test F1 Score: 0.6460
Test Confusion Matrix:
[[  0   1  12]
 [  0   5  36]
 [  0  10 141]]

Running Decision Tree model...
Best Params: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 30}
Decision Tree Model Metrics:
Train Accura

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.8866
Train Precision: 0.8906
Train Recall: 0.8866
Train F1 Score: 0.8813
Train Confusion Matrix:
[[ 23   5   4]
 [  0  92  40]
 [  0   5 307]]

Test Accuracy: 0.6976
Test Precision: 0.6720
Test Recall: 0.6976
Test F1 Score: 0.6835
Test Confusion Matrix:
[[  1   3   9]
 [  4  14  23]
 [  2  21 128]]

Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9834
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 127   5]
 [  0   1 311]]

Test Accuracy: 0.7317
Test Precision: 0.6876
Test Recall: 0.7317
Test F1 Score: 0.7085
Test Confusion Matrix:
[[  0   3  10]
 [  1  18  22]
 [  2  17 132]]

Running AdaBoost model...


...

Best Params: {'learning_rate': 0.3, 'iterations': 150, 'depth': 4}
CatBoost Model Metrics:
Train Accuracy: 0.9832
Train Precision: 0.9834
Train Recall: 0.9832
Train F1 Score: 0.9831
Train Confusion Matrix:
[[ 30   0   2]
 [  0 127   5]
 [  0   1 311]]

Test Accuracy: 0.7024
Test Precision: 0.6412
Test Recall: 0.7024
Test F1 Score: 0.6690
Test Confusion Matrix:
[[  0   6   7]
 [  2  10  29]
 [  0  17 134]]

Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.7311
Train Precision: 0.7266
Train Recall: 0.7311
Train F1 Score: 0.6949
Train Confusion Matrix:
[[  6   8  18]
 [  0  48  84]
 [  1  17 294]]

Test Accuracy: 0.7122
Test Precision: 0.6570
Test Recall: 0.7122
Test F1 Score: 0.6747
Test Confusion Matrix:
[[  1   2  10]
 [  2   9  30]
 [  1  14 136]]

Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.8845
Train Precision: 0.8938
Train Recall: 0.8845
Train F1 Score: 0.8786
Train Confusion Matrix:
[[ 24   1   7]
 [  0  89  43]
 [  1   3 308]]

Test Accuracy: 0.6683
Test Precision: 0.6342
Test Recall: 0.6683
Test F1 Score: 0.6483
Test Confusion Matrix:
[[  1   3   9]
 [  2  10  29]
 [  2  23 126]]

